Define our prompt

In [3]:
prompt = """
Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not be a list, it should be a JSON object.
${gr.complete_json_suffix}

an example of output may look like this:
{
	"user_orders": [{                                                                                                   │ │
        "user_id": 1,
        "user_name": "John Mcdonald",
    	"num_orders": 6
    }]
}

"""

Install validators

In [4]:
!guardrails hub install hub://guardrails/valid_length
!guardrails hub install hub://guardrails/two_words
!guardrails hub install hub://guardrails/valid_range

2024-03-25 16:13:45 zmac guardrails-cli[96111] NOTICE Installing hub://guardrails/valid_length...
  Running command git clone --filter=blob:none --quiet https://github.com/guardrails-ai/valid_length.git /private/var/folders/c8/jqt82fpx785dpwpp36ljkgm40000gn/T/pip-req-build-y3acxka5
2024-03-25 16:13:52 zmac guardrails-cli[96111] INFO Collecting git+https://github.com/guardrails-ai/valid_length.git
  Cloning https://github.com/guardrails-ai/valid_length.git to /private/var/folders/c8/jqt82fpx785dpwpp36ljkgm40000gn/T/pip-req-build-y3acxka5
  Resolved https://github.com/guardrails-ai/valid_length.git to commit 4b59a5fc1ae2106a585881784e3c2086f1fe8b9b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (p

Defined what our data should look like in pydantic

In [5]:
from pydantic import BaseModel, Field
from guardrails.hub import ValidLength, TwoWords, ValidRange
from typing import List

class Order(BaseModel):
	user_id: int = Field(description="The user's id.", validators=[("1-indexed", "noop")])
	user_name: str = Field(
		description="The user's first name and last name",
		validators=[TwoWords()]
	)

	num_orders: int = Field(
		description="The number of orders the user has placed",
		validators=[ValidRange(0, 50)]
	)
	


class Orders(BaseModel):
	user_orders: List[Order] = Field(
		description="Generate a list of users and how many orders they have placed in the past.",
		validators=[ValidLength(10, 10, on_fail="noop")]
	)

Set up the cohere client. In this notebook, we use an environment variable to pass the cohere API key through

In [6]:
import cohere
import os

api_key = os.environ['COHERE_API_KEY']

co = cohere.Client(api_key=api_key)

Create the guard and prompt

In [7]:
import guardrails as gd
guard = gd.Guard.from_pydantic(output_class=Orders, prompt=prompt)

res = co.chat(message="hi")

raw_llm_response, validated_response, *rest = guard(
	co.chat,
	model="command",
	max_tokens=1024,
	temperature=0.3
)

/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.11/site-packages/guardrails/validatorsattr.py:307: UserWarning: Validator 1-indexed is not installed!
  warnings.warn(f"Validator {validator_name} is not installed!")
/Users/zaydsimjee/workspace/guardrails/.venv/lib/python3.11/site-packages/guardrails/validators/__init__.py:50: FutureWarning: 
    Importing validators from `guardrails.validators` is deprecated.
    All validators are now available in the Guardrails Hub. Please install
    and import them from the hub instead. All validators will be
    removed from this module in the next major release.

    Install with: `guardrails hub install hub://<namespace>/<validator_name>`
    Import as: from guardrails.hub import `ValidatorName`
    
  warn(

HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
Diffusion not supported. Skipping import.
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"


By printing the guard's state, we can see the final, validated output, as well as the steps guardrails took to ensure structure.

In [8]:
from rich import print

print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not  │ │
    │ │ be a list, it should be a JSON object.                                                                  │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <list name="user_orders" description="Generate a list of users and how many orders they have placed │ │
    │ │ in the past." format="guardrails/valid_length: min=10 max=10">                                          │ │
    │ │         <object>                                                                                        │ │
    │ │             <integer name="user_id" description="The user's id." format="1-indexed"/>                   │ │
    │ │             <string name="user_name" description="The user's first name and last name"                  │ │
    │ │ format="guardrails/two_words"/>                                                                         │ │
    │ │             <integer name="num_orders" description="The number of orders the user has placed"           │ │
    │ │ format="guardrails/valid_range: min=0 max=50"/>                                                         │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz':